<a href="https://colab.research.google.com/github/gokul-gk17/cse326/blob/main/PaddleOCR_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install paddlepaddle paddleocr jiwer pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=662522479d5139d8a28ce313334b7d2d0b93056eba75de30c990643ee024a9b9
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
  Attempting uninstall: opt_einsum
    Found ex

In [5]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from jiwer import wer, cer
import os
import numpy as np


In [1]:
!pip install paddlepaddle==2.6.2 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html



Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [2]:
!pip install "paddleocr>=2.6.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=e9cfd75e0f5383c4a116bea97b450d046362cb0c51c26e678663d4e8abf65008
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [7]:
!pip install jiwer pandas opencv-python-headless scikit-learn

In [8]:
import os, time, re
import pandas as pd
import numpy as np
import cv2
from paddleocr import PaddleOCR
from sklearn.metrics import precision_score, recall_score, f1_score
from jiwer import wer, cer

In [9]:
ocr = PaddleOCR(use_angle_cls=True, lang='en')


IMAGE_DIR = "/content/drive/MyDrive/OCR_Project/test_images"
RESULTS_PATH = "/content/drive/MyDrive/OCR_Project/PaddleOCR_Results/results.csv"
VIS_DIR = "/content/drive/MyDrive/OCR_Project/PaddleOCR_Results/vis"
os.makedirs(VIS_DIR, exist_ok=True)

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


def get_sorted_pairs(image_dir):
    files = []
    for f in os.listdir(image_dir):
        if f.lower().endswith(('png', 'jpg', 'jpeg')):
            base = os.path.splitext(f)[0]
            txt_path = os.path.join(image_dir, f"{base}.txt")
            if os.path.exists(txt_path):
                files.append((f, base))
    files.sort(key=lambda x: natural_sort_key(x[1]))
    return [(os.path.join(image_dir, f), os.path.join(image_dir, f"{base}.txt")) for f, base in files]


def calculate_metrics(gt, pred):
    gt_words = set(gt.split())
    pred_words = set(pred.split())
    all_words = list(gt_words.union(pred_words))
    y_true = [1 if w in gt_words else 0 for w in all_words]
    y_pred = [1 if w in pred_words else 0 for w in all_words]
    return {
        'Accuracy': int(gt == pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'CER': cer(gt, pred),
        'WER': wer(gt, pred)
    }


results = []
for img_path, txt_path in get_sorted_pairs(IMAGE_DIR):
    with open(txt_path, 'r') as f:
        gt_text = f.read().strip()
    try:
        start = time.time()
        result = ocr.ocr(img_path, cls=True)
        time_taken = time.time() - start

        pred_text = ' '.join([word_info[1][0] for block in result for word_info in block]).strip()

        img = cv2.imread(img_path)
        for block in result:
            for word_info in block:
                if len(word_info) >= 1:
                    bbox = word_info[0]
                    pts = [(int(x), int(y)) for x, y in bbox]
                    cv2.polylines(img, [np.array(pts)], isClosed=True, color=(0, 255, 0), thickness=2)

        vis_path = os.path.join(VIS_DIR, os.path.basename(img_path))
        cv2.imwrite(vis_path, img)

        metrics = calculate_metrics(gt_text, pred_text)
        metrics.update({'Image': os.path.basename(img_path), 'Time': time_taken})
        results.append(metrics)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

pd.DataFrame(results).to_csv(RESULTS_PATH, index=False)
print("PaddleOCR evaluation complete!")


[2025/04/12 14:44:48] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_l